<a href="https://colab.research.google.com/github/swapnilkhartode/LGMVIP-Data-Science/blob/README/Advanced%20Level%20Tasks/Task%3A%202_Next_Word_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **LetsGrowMore Virtual Internship Program (November-2021)**

**Advanced Level Task**

Name :- Swapnil Ramesh Khartode

Task 2 :- Next Word Prediction

**Import the required libraries**

In [ ]:
import numpy as np
from nltk.tokenize import RegexpTokenizer
from keras.models import Sequential, load_model
from keras.layers import LSTM
from keras.layers.core import Dense, Activation
from tensorflow.keras.optimizers import RMSprop
import matplotlib.pyplot as plt
import pickle
import heapq

**Load the data**

In [ ]:
text = open('/content/1661-0.txt').read().lower()

In [ ]:
print('Length of the text:', len(text))

Length of the text: 581888


**Split the dataset**

In [ ]:
tokenizer = RegexpTokenizer(r'\w+')
words = tokenizer.tokenize(text)

In [ ]:
unique_words = np.unique(words)
unique_words_index = dict((c, i) for i, c in enumerate(unique_words))

In [ ]:
word_length = 5
prev_words = []
next_words = []
for i in range(len(words) - word_length):
    prev_words.append(words[i:i + word_length])
    next_words.append(words[i + word_length])
print(prev_words[0])
print(next_words[0])

['project', 'gutenberg', 's', 'the', 'adventures']
of


In [ ]:
dataX = np.zeros((len(prev_words), word_length, len(unique_words)), dtype=bool)
dataY = np.zeros((len(next_words), len(unique_words)), dtype=bool)
for i, each_words in enumerate(prev_words):
    for j, each_word in enumerate(each_words):
        dataX[i, j, unique_words_index[each_word]] = 1
    dataY[i, unique_words_index[next_words[i]]] = 1

**Build the RNN (Recurrent Neural Network)**

In [ ]:
model = Sequential()
model.add(LSTM(128, input_shape=(word_length, len(unique_words))))
model.add(Dense(len(unique_words)))
model.add(Activation('softmax'))

**Train the model**

In [ ]:
optimizer = RMSprop(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
history = model.fit(dataX, dataY, validation_split=0.05, batch_size=128, epochs=3, shuffle=True).history
#epochs=30 gives more accurate prediction

Epoch 1/3
811/811 [==============================] - 308s 378ms/step - loss: 3.4168 - accuracy: 0.6001 - val_loss: 10.0284 - val_accuracy: 0.0657
Epoch 2/3
811/811 [==============================] - 303s 373ms/step - loss: 3.2491 - accuracy: 0.6157 - val_loss: 10.0879 - val_accuracy: 0.0622
Epoch 3/3
811/811 [==============================] - 306s 378ms/step - loss: 3.2012 - accuracy: 0.6240 - val_loss: 10.3796 - val_accuracy: 0.0615


In [ ]:
model.save('next_word_model.h5')
pickle.dump(history, open("history.p", "wb"))
model = load_model('next_word_model.h5')
history = pickle.load(open("history.p", "rb"))

**Test the model**

In [ ]:
def prepare_input(text):
    x = np.zeros((1, word_length, len(unique_words)))
    for t, word in enumerate(text.split()):
        print(word)
        x[0, t, unique_words_index[word]] = 1
    return x
prepare_input("It is not a lack".lower())

it
is
not
a
lack


array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]])

In [ ]:
#to return the samples
def return_sample(preds, top_n=3):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds)
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)

    return heapq.nlargest(top_n, range(len(preds)), preds.take)

In [ ]:
def predict_words(text, n=3):
    x = prepare_input(text)
    preds = model.predict(x, verbose=0)[0]
    next_indices = return_sample(preds, n)
    return [indices_char[idx] + predict_words(text[1:] + indices_char[idx]) for idx in next_indices]